In [1]:
import pyrealsense2 as rs
import matplotlib.pyplot as plt    
import scipy
from sklearn.neighbors import NearestNeighbors
import src.realsense.dataset as rld
import src.realsense.camera as rlc
import src.visual3d.geometry as geom
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import cv2

%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv("data/colours_rgb_shades.csv", usecols = [1,2,3,4])
data.head()

numpied_data = data.to_numpy()[:,:3]
# print("Numpied Data : ", numpied_data)

numpied_vals = data.to_numpy()[:,-1:].reshape(-1)
# print("NUmpied Values : ", numpied_vals)

model = RandomForestClassifier()
model.fit(numpied_data, numpied_vals)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [3]:
# GMM function

import itertools
from scipy import linalg
import matplotlib as mpl

from sklearn import mixture



color_iter = itertools.cycle(["navy", "c", "cornflowerblue", "gold", "darkorange"])


def plot_results(X, Y_, means, covariances, index, title):
    splot = plt.subplot(1, 1, 1 + index)
    for i, (mean, covar, color) in enumerate(zip(means, covariances, color_iter)):
        v, w = linalg.eigh(covar)
        v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
            
        if v[1]/v[0]<3.5:
            continue
    
#         plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], 0.8, color=color)
        
        p0 = mean - np.array([u[0],u[1]])*v[0]/(2.0 )
        p1 = mean + np.array([u[1],u[1]])*v[0]/(2.0 )
        p2 = mean - np.array([u[1],-u[0]])*v[1]/(2.0 )
        p3 = mean + np.array([u[1],-u[0]])*v[1]/(2.0 )    
        plt.plot([p2[0],p3[0]],[p2[1],p3[1]],color=color)
        print("Points : ", [p2[0],p3[0]],[p2[1],p3[1]])
        
        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180.0 * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180.0 + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.2)
        splot.add_artist(ell)
#         plt.axis('equal')
#         plt.xlim([0,1280])
#         plt.ylim([0,720])


#     plt.title(title)
    


In [8]:
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

fps = 60
W = 640
H = 480
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, fps)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, fps)


# Start streaming
profile = pipeline.start(config)


# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)


# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)
count = 0
# Streaming loop
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()
        
        # Get the intrinsics
        depth_intrin = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
        K = np.array([[depth_intrin.fx, 0, depth_intrin.ppx],
                      [0, depth_intrin.fy, depth_intrin.ppy],
                      [0, 0, 1]])
                    
#         print("INTRINSICS : ", K)

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        if count % 10 ==0:
                    

            depth = depth_image
            color = color_image

            xyz,rgb = rlc.project_rgbd2pcd(color,depth,K) # project to point cloud
            idx = xyz[:,2]>0.001
            xyz = xyz[idx,:]; rgb = rgb[idx,:]

            xy = xyz[:,:2]; z=xyz[:,2]
            reg = linear_model.LinearRegression()
            reg.fit(xy,z)

            print(f"plane: z = {reg.coef_[0]}x + {reg.coef_[1]}y + {reg.intercept_} ")
            class_binary = (z - (xy @ reg.coef_ + reg.intercept_) ) < -0.030

            xyz_top = xyz[class_binary,:]
            rgb_top = rgb[class_binary,:]
            xyz_bottom = xyz[~class_binary,:]
            rgb_bottom = rgb[~class_binary,:]

            color_top, depth_top = rlc.project_pcd2rgb(xyz_top,rgb_top,H,W,K)


            x_idx = np.array([range(W) for _ in range(H)])
            y_idx = np.array([range(H) for _ in range(W)]).T
            binary_depth = depth_top > 





            0.001
            gmm_x = x_idx[binary_depth].reshape(-1)
            gmm_y = y_idx[binary_depth].reshape(-1)
            gmm_xy = np.c_[gmm_x,gmm_y]

            X = gmm_xy[::20,:]
            dpgmm = mixture.BayesianGaussianMixture(n_components=30, covariance_type="full").fit(X)

            lines = []
            for i, (mean, covar) in enumerate(zip(dpgmm.means_,  dpgmm.covariances_)):
                v, w = linalg.eigh(covar)
                v = np.sqrt(2.0) * np.sqrt(v)
                u = w[0] / linalg.norm(w[0])
                if not np.any(dpgmm.predict(X) == i):
                    continue
                if v[1]/v[0]<3.5:
                    continue
                p2 = mean - np.array([u[1],-u[0]])*v[1]
                p3 = mean + np.array([u[1],-u[0]])*v[1]  
                lines.append((p2,p3))

            intersections = []
            vectors1 = []
            vectors2 = []
            vectors3d_1 = []
            vectors3d_2 = []
            end_v1 = []
            end_v2 = []
            for i in range(len(lines)):
                for j in range(i+1,len(lines)):
                    (x1,y1),(x2,y2) = lines[i]
                    (x3,y3),(x4,y4) = lines[j]
                    px = ((x1*y2-y1*x2)*(x3-x4) - (x1-x2)*(x3*y4-y3*x4)) / ((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))
                    py = ((x1*y2-y1*x2)*(y3-y4) - (y1-y2)*(x3*y4-y3*x4)) / ((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))   
                    thres = 100
                    m1 = (y1-y2)/(x1-x2)
                    m2 = (y3-y4)/(x3-x4)
                    if px < max(min(x1,x2),min(x3,x4))-thres or px > min(max(x1,x2),max(x3,x4))+thres:
                        continue


                    elif 0 < abs(m1-m2) and abs(m1-m2) < 0.16:

                        continue

    #                 elif px < color.shape[1] and py < color.shape[0] and int(model.predict(np.array(color[int(py), int(px), :]).reshape(1,3))) == 0:

    #                     continue

                    else:
                        # 2d space
                        intersections.append([px,py])
                        v1 = np.array([x2-x1,y2-y1]); v1 = v1/np.linalg.norm(v1)
                        v2 = np.array([x4-x3,y4-y3]); v2 = v2/np.linalg.norm(v2)
                        vectors1.append(v1); vectors2.append(v2)

            intersections = np.array(intersections)

            thickness = 2
            for (px,py),(v1x,v1y),(v2x,v2y) in zip(intersections,vectors1,vectors2):


                color = cv2.arrowedLine(color, (int(px),int(py)), (int(px+10*v1x),int(py+10*v1y)), color=(255,0,0), thickness=3)
                color = cv2.arrowedLine(color, (int(px),int(py)), (int(px+10*v2x),int(py+10*v2y)), color=(255,0,0), thickness=3)


    #         plt.figure(figsize=(11,8), dpi= 100, facecolor='w', edgecolor='k')
    #         plt.scatter(intersections[:,0],intersections[:,1],100)
    #         for (px,py),(v1x,v1y),(v2x,v2y) in zip(intersections,vectors1,vectors2):
    #             vlength = 100
    #             plt.arrow(px,py,vlength*v1x,vlength*v1y,head_width=30)
    #             plt.arrow(px,py,vlength*v2x,vlength*v2y,head_width=30)


    #         plt.gca().invert_yaxis()

    #         plt.subplot(111)
    #         plt.imshow(color)


    #         depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
    #         images = np.hstack((color_image, depth_colormap))

            cv2.namedWindow('Align Example', cv2.WINDOW_NORMAL)
            cv2.imshow('Align Example', color)
            key = cv2.waitKey(1)
            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                break
finally:
    pipeline.stop()

Depth Scale is:  0.0010000000474974513
plane: z = -0.08893330381165454x + 0.35606653480989775y + 2514.7780655055453 


/usr/lib/python3/dist-packages/sklearn/mixture/_base.py:263: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


plane: z = -0.08704000550158776x + 0.3569114501520903y + 2516.701417170389 


/usr/lib/python3/dist-packages/sklearn/mixture/_base.py:263: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


plane: z = -0.09215046769579538x + 0.34849671393929915y + 2523.7868938400725 


/usr/lib/python3/dist-packages/sklearn/mixture/_base.py:263: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


plane: z = -0.09699264965477541x + 0.343256372470601y + 2524.6736030061957 


/usr/lib/python3/dist-packages/sklearn/mixture/_base.py:263: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


plane: z = -0.27395210605834674x + 0.0010805560687892296y + 2605.2108005651094 


/usr/lib/python3/dist-packages/sklearn/mixture/_base.py:263: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
